In [ ]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
import nltk
import re
import string
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
import heapq
import yaml
from nltk.stem import WordNetLemmatizer, PorterStemmer

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from scipy.special import softmax
import numpy as np
from array import array
import copy

import time

start = time.time()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")

def test_entailment(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.no_grad():
        proba = torch.softmax(model(**batch).logits, -1)
    return proba.cpu().numpy()[0, model.config.label2id['ENTAILMENT']]

def test_equivalence(text1, text2):
    return test_entailment(text1, text2) * test_entailment(text2, text1) * 100

In [ ]:
questions = pd.read_csv('questions.csv')
display(questions)

In [ ]:
questions = questions.drop(columns=['id', 'qid1', 'qid2'])
display(questions)

In [ ]:
test_questions = questions.copy()
test_questions

In [ ]:
from tqdm import tqdm

resulting_df = pd.DataFrame(columns=['question1', 'question2', 'result', 'transcribed_result'])

for index, row in tqdm(test_questions.iterrows(), total=test_questions.shape[0]):
    question_a = row['question1']
    question_b = row['question2']
    
    result = test_equivalence(str(question_a), str(question_b))
    
    transcribed_result = 0
    if result > 1 :
        transcribed_result = 1
    
    resulting_df = resulting_df.append({'question1': question_a, 'question2': question_b, 'result': result, 'transcribed_result': transcribed_result}, ignore_index=True)
    
resulting_df.to_csv('questions-bert-similarity-full.csv')

In [ ]:
resulting_df

In [ ]:
test_questions = test_questions.reset_index(drop=True)
test_questions['is_match'] = test_questions['is_duplicate'] == resulting_df['transcribed_result']
test_questions['is_match'] = test_questions['is_match'].astype(int)

test_questions.to_csv('questions-roberta-similarity-with-match.csv')

In [ ]:
count_ones = (test_questions['is_match'] == 1).sum()
total_count = len(test_questions['is_match'])
percentage_ones = (count_ones / total_count) * 100

print("Percentage of 1s: {:.2f}%".format(percentage_ones))